# Generate Feature Matrix

In [1]:
from DOLPHIN import combine_fea, fea_comp, get_fea_hvg

/mnt/md0/kailu/anaconda3/envs/DOLPHIN/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
metadata = "your_metaData.csv"
df_label = pd.read_csv(metadata, sep='\t')
total_sample_size = len(df_label)

# Define your study name (update this for your specific study)
study_name = "STUDY"

fea_run_num = 100

output_directory = os.path.join(main_folder, "final_data")
graph_directory = os.path.join(main_folder "06_graph_mtx")
exon_gene_featurecount_directory=os.path.join(main_folder, "04_exon_gene_cnt")

# Required files
gene_annotation = "gene_id_annotation.csv"
gtf_pkl_path = "gtf.pkl"
gtf_jun_pkl_path = "df_jun_gtf.pkl"
gtf_path= "Homo_sapiens.GRCh38.107.gtf"

In [ ]:
print("Start Combine Feature Matrix")
with tqdm(total=total_sample_size) as pbar_fea:
    for i in range(0, total_sample_size):
        if i%fea_run_num==0:
            pbar_fea=combine_fea(pbar_fea, df_label, graph_directory, gene_annotation, gtf_pkl_path, start_idx = i, sample_num=fea_run_num, output_path=output_directory, output_name= study_name)

In [ ]:
#### combine all feature .h5ad files
total_number_fea_anndata = math.ceil(total_sample_size/fea_run_num)
for _idx, _fea_idx in enumerate(range(0, total_number_fea_anndata)):
    _temp_ad = anndata.read_h5ad(os.path.join(output_directory, "Feature_"+study_name+"_"+str(_fea_idx)+".h5ad"))
    if _idx ==0:
        combine_anndata_fea = _temp_ad
    else:
        combine_anndata_fea = combine_anndata_fea.concatenate(_temp_ad, index_unique = None, batch_key = None)
combine_anndata_fea.write(os.path.join(output_directory,"Feature_"+study_name+".h5ad"))

fea_comp(output_directory, study_name)

In [ ]:
### Remove All Temporary Datasets
file_pattern = os.path.join(output_directory, f"Feature_{study_name}_*")

files_to_remove = glob.glob(file_pattern)

# Remove each file
for file in files_to_remove:
    if os.path.exists(file):  # Check if the file exists
        os.remove(file)
        print(f"Removed: {file}")
    else:
        print(f"File not found: {file}")

### Filter Feature Matrix with Highly Variable Genes (HVG)

In [ ]:
get_fea_hvg(output_directory, study_name)